In [4]:
# from txt files extracted using imageJ plugin,
# process data, correct for crosstalk and direct acceptor excitation
# select data with only 1 donor and 1 acceptor molecule and no bleaching
# add de-noising step using chung kennedy filtering
# calculate correlation between donor and acceptor to know whether the traces is static or dynamic fret
# for static traces, check if it is low or high FRET
# make .out file with donor and acceptor values
# make pdf with donor acceptor (including data before and after filtering), sum D+A and FRET

In [5]:
import numpy as np
from scipy.signal import medfilt as filt_med
from scipy.ndimage import uniform_filter1d as filt_mean
import matplotlib.pyplot as plt
import os
import os.path as path
import seaborn as sns

sns.set(font="Arial")
sns.set_style("ticks")
sns.set_context("paper", font_scale=1.45,rc={"grid.linewidth": 0,"axes.linewidth": 1})

filtersize_signal = 3 # number of frames, should be odd
filtersize_bg = 7 # number of frames, should be odd
frametime = 0.2 # seconds
left2right_bleed = 0.071 # fraction of left channel visible in right channel
exc_bleed = 0.078
# position constraints
enable_position_constraints = True
x_min = 10  # minimum value of x_pos (px)
x_max = 235 # maximum value of x_pos (px)
y_min = 10 # minimum value of y_pos (px)
y_max = 502 # maximum value of y_pos (px)
# intensity constraints
enable_intensity_constraints = True
signal_right1_med_start  = 1   # first frame for average
signal_right2_med_start  = 476   # first frame for average
signal_length1 = 25 # number of frames to average
signal_length2 = 450
signal_corr_start = 25
signal_right_min = 20000    # minimum value of average
signal_right_max = 40000 # maximum value of average
signal_sum1_med_start  = 26  # first frame for average
signal_sum2_med_start  = 451  # first frame for average
signal_left1_med_start = 26
signal_left2_med_start = 451
signal_left_min = 0
signal_left_max = 25000
signal_sum_min = 10000    # minimum value of average
signal_sum_max = 40000 # maximum value of average
# Dynamic coonstraints
enable_dynamic_constraints = True
corr_th = -0.01
# FRET constraints
enable_fret_constraints = True
FRETth = 0.4

def odd(f):
	return f + 1 if f % 2 == 0 else f
# forward and backward estimates of the datapoint x_f(k) and x_b(k)
def fw_est(data, k, width):
    if (k>0):
        start = max(0, k-width)
        finish = k-1
        return np.mean(data[start:finish+1])
    else: 
        return data[k]

def bw_est(data, k, width):
    if k<len(data)-1:
        start = k+1
        finish = min(k+width, len(data))
        return np.mean(data[start:finish+1])
    else:
        return data[k]
# non-normalized weights for the estimates f(k) and b(k)
def fw_weight_nonnorm (data, k, M, p, width):
    tmp=0
    for j in range(max(0,k-M+1),k):
        tmp = tmp + (data[j]-fw_est(data, j, width))**(2*p)
    return 1.0/(tmp+10**-10)

def bw_weight_nonnorm (data, k, M, p, width):
    tmp=0
    for j in range(k+1, min(k+M, len(data))):
        tmp = tmp + (data[j]-bw_est(data, j, width))**(2*p)
    return 1.0/(tmp+10**-10)

# Chung-Kennedy filtering
def ChungKennedy (data, k, KC, M, p):
    top = 0
    bottom = 0
    for i in range(1,KC+1):
        top = top + fw_weight_nonnorm(data, k, M, p, i)*fw_est(data,k,i)+bw_weight_nonnorm(data, k, M, p, i)*bw_est(data,k,i)
        bottom = bottom + fw_weight_nonnorm(data, k, M, p, i) +  bw_weight_nonnorm(data, k, M, p, i)
    return top/(bottom + 10**-10)

def process_file(dirname, filename):
	(x_pos, y_pos) = [int(i) for i in filename.split('_')[4].split(',')]
	# check position constraints, and return if not met
	if enable_position_constraints and (x_pos < x_min or x_pos > x_max or y_pos < y_min or y_pos > y_max):
		return
	# load data
	filepath = path.join(dirname, filename)
	name = path.splitext(filename)[0]
	area_left = 0
	area_left_bg = 0
	area_right = 0
	area_right_bg = 0
	with open(filepath) as file:
		last_comment = ''
		while True:
			line = file.readline()
			if line.startswith('#'):
				last_comment = line
				continue
			else:
				break
		last_comment = last_comment[1:]
		(area_left, area_left_bg, area_right, area_right_bg) = [int(i) for i in last_comment.split()]
	data = np.loadtxt(filepath).transpose()
	left    = data[0]
	leftBg  = data[1]
	right   = data[2]
	rightBg = data[3]
	# process data
	t = np.arange(left.size) * frametime
	left_bg_filt  = filt_med(leftBg,  odd(filtersize_bg))
	right_bg_filt = filt_med(rightBg, odd(filtersize_bg))
	signal_left  = (left  - left_bg_filt) * float(area_left)
	signal_right = (right - right_bg_filt) * float(area_right)
	signal_left_filt  = filt_med(signal_left,  odd(filtersize_signal))
	signal_right_filt_uncorr = filt_med(signal_right, odd(filtersize_signal))
	signal_right_filt_corr = signal_right_filt_uncorr - (left2right_bleed * signal_left_filt) - (exc_bleed * signal_right_filt_uncorr)
	signal_sum = np.array(signal_left_filt) + np.array(signal_right_filt_corr)
	signal_fret = np.array(signal_right_filt_corr) / np.array(signal_sum)
	signal_fret = np.maximum(signal_fret, -1)
	signal_fret = np.minimum(signal_fret, 2)
	#check intensity constraints, and return if not met
	signal_right1_average = np.median(signal_right_filt_corr[signal_right1_med_start:signal_right1_med_start+signal_length1])
	if enable_intensity_constraints and (signal_right1_average < signal_right_min or signal_right1_average > signal_right_max):
		return
	signal_sum1_average = np.median(signal_sum[signal_sum1_med_start:signal_sum1_med_start+signal_length1])
	if enable_intensity_constraints and (signal_sum1_average < signal_sum_min or signal_sum1_average > signal_sum_max):
		return
	signal_left1_average = np.median(signal_left_filt[signal_left1_med_start:signal_left1_med_start+signal_length1])
	if enable_intensity_constraints and (signal_left1_average < signal_left_min or signal_left1_average > signal_left_max):
		return
	signal_sum2_average = np.median(signal_sum[signal_sum2_med_start:signal_sum2_med_start+signal_length1])
	if enable_intensity_constraints and (signal_sum2_average < signal_sum_min or signal_sum2_average > signal_sum_max):
		return
	signal_right2_average = np.median(signal_right_filt_corr[signal_right2_med_start:signal_right2_med_start+signal_length1])
	if enable_intensity_constraints and (signal_right2_average < signal_right_min or signal_right2_average > signal_right_max):
		return
	signal_left2_average = np.median(signal_left_filt[signal_left2_med_start:signal_left2_med_start+signal_length1])
	if enable_intensity_constraints and (signal_left2_average < signal_left_min or signal_left2_average > signal_left_max):
		return
	signal_right_ck = [ChungKennedy(signal_right_filt_corr, i, 10, 10, 1)for i in range(len(signal_right_filt_corr))]
	signal_left_ck = [ChungKennedy(signal_left_filt, i , 10, 10, 1)for i in range(len(signal_left_filt))]
	signal_sum_ck = np.array(signal_left_ck) + np.array(signal_right_ck)
	signal_fret_ck = np.array(signal_right_ck) / np.array(signal_sum_ck)
	signal_fret_ck = np.maximum(signal_fret_ck, -1)
	signal_fret_ck = np.minimum(signal_fret_ck, 2)
	# check if dynamic
	dD = np.array(signal_left_ck[signal_corr_start:signal_corr_start+signal_length2]) - np.mean(np.array(signal_left_ck[signal_corr_start:signal_corr_start+signal_length2]))
	dA = np.array(signal_right_ck[signal_corr_start:signal_corr_start+signal_length2]) - np.mean(np.array(signal_right_ck[signal_corr_start:signal_corr_start+signal_length2]))
	corr = np.mean(dD * dA)/(np.mean(np.array(signal_left_ck[signal_corr_start:signal_corr_start+signal_length2]) + np.array(signal_right_ck[signal_corr_start:signal_corr_start+signal_length2]))**2)  
	if enable_dynamic_constraints and (corr < corr_th): 
		OutName = 'D' + name
	else: 
		OutName = 'S' + name
	# check is Static traces are low or high FRET    
	MeanFRET = np.median(signal_fret_ck[signal_corr_start:signal_corr_start+signal_length2])
	if enable_fret_constraints and (MeanFRET > FRETth):
		OutName = 'FRET_' + OutName
	else: 
		OutName = OutName
	# save filtered signals left and right to txt file
	signal_out = np.column_stack((signal_left_ck, signal_right_ck))
	np.savetxt(path.join(dirname, OutName + '.out.txt'), signal_out, fmt='%.5e', delimiter='\t')
	# plot data to pdf file
	fig, (plt1, plt2) = plt.subplots(2, 1,figsize=(5, 3))
	plt1.plot(t, signal_left_filt,  'green')
	plt1.plot(t, signal_right_filt_corr, 'red')
	plt1.plot(t, signal_left_ck,  'black',linewidth=0.5)
	plt1.plot(t, signal_right_ck, 'black',linewidth=0.5)
	plt1.set(ylabel='Intensity (a.u.)')
	plt1.set_xlim(left=0, right=100)
	plt1.set_xticklabels([])
	plt1.axis([0,100,-5000, 39000])
	plt2.plot(t, signal_fret, 'blue')
	plt2.plot(t, signal_fret_ck, 'black',linewidth=0.5)
	plt2.text(-19,0.45, "FRET Efficiency", ha="center", va="center", rotation=90)
	plt2.set(xlabel='Time (s)')
	plt2.axis([0,100,-0.1, 1.1])
	fig.tight_layout(pad=0)
	fig.savefig(path.join(dirname, name + '.out.svg'),bbox_inches='tight')
	plt.close(fig)

def print_progress_bar(iteration, total, prefix = '', suffix = '', decimals = 1, length = 50, fill = '█', print_end = '\r'):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filled_length = int(length * iteration // total)
    bar = fill * filled_length + '-' * (length - filled_length)
    print(f'\r{prefix}|{bar}| {percent}%{suffix}', end = print_end)
    # print new line on complete
    if iteration == total: 
        print()

# get current dir
dirname = os.getcwd()
# ignore division by zero warning
np.seterr(divide='ignore', invalid='ignore')
# get valid files in dirname
filenames = []
for filename in os.listdir(dirname):
	(name, ext) = path.splitext(filename)
	if path.isfile(path.join(dirname, filename)) and ext == '.txt' and not name.endswith('.out'):
		filenames.append(filename)
if len(filenames) == 0:
	print('No files to process')
	exit()
# process files
print_progress_bar(0, len(filenames), prefix = 'Progress: ')
for i, filename in enumerate(filenames):
	process_file(dirname, filename)
	print_progress_bar(i + 1, len(filenames), prefix = 'Progress: ')

Progress: |██████████████████████████████████████████████████| 100.0%
